In [1]:
import json, pathlib, pandas as pd
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
import os, csv
import cProfile, pstats
from persistent_homology import (
    BettiZero,
    compute_intervals,
    compute_n_largest_bars,
    generate_sphere_points,
)

In [2]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output
    
def get_subfolders(path):
    """
    Return a list of names of all subdirectories in the given path.
    """
    return [
        name for name in os.listdir(path)
        if os.path.isdir(os.path.join(path, name))
    ]

def load_vertices_edges(seg_folder):
    """Read vertices/edges only once per segmentation."""
    folder_path = '../lung_segmentations/{}'.format(seg_folder)
    edges_path = '/edges.csv'
    vertices_path = '/vertices.csv'
    verts = read_csv(folder_path + vertices_path)
    edges = read_csv(folder_path + edges_path)
    return verts, edges

def process_direction(args):
    """Run β₀ persistence for one direction (runs in worker)."""
    direction, vertices, edges = args
    bz = BettiZero(direction, vertices, edges)
    comps, mergers, verts, births = bz.compute_persistence()
    intervals = compute_intervals(births, mergers)
    bars = compute_n_largest_bars(intervals,5)
    return {
        "direction": list(direction),
        "intervals": intervals,
        "largest_bars": bars,
        #"largest_length": length,
        "components": list(comps),
    }

In [3]:
json_data = {}

segmentation_folder_path = "../lung_segmentations"
lung_segmentations = get_subfolders(segmentation_folder_path)

In [4]:
vertices, edges = load_vertices_edges('Lung segmentation-10')
directions = generate_sphere_points(2, 2, 1e-7)

In [5]:
process_direction(([0,0,1], vertices, edges))

{'direction': [0, 0, 1],
 'intervals': [[93.5, 'infty'],
  [94.5, 98.5],
  [95.5, 112.5],
  [96.5, 103.5],
  [100.5, 108.5],
  [102.5, 103.5],
  [103.5, 129.5],
  [103.5, 107.5],
  [106.5, 107.5],
  [107.5, 113.5],
  [107.5, 127.5],
  [113.5, 123.5],
  [114.5, 170.5],
  [116.5, 119.5],
  [120.5, 'infty'],
  [120.5, 149.5],
  [121.5, 123.5],
  [123.5, 'infty'],
  [125.5, 126.5],
  [126.5, 144.5],
  [129.5, 132.5],
  [130.5, 131.5],
  [132.5, 145.5],
  [132.5, 133.5],
  [133.5, 134.5],
  [136.5, 137.5],
  [137.5, 139.5],
  [140.5, 146.5],
  [140.5, 145.5],
  [140.5, 141.5],
  [141.5, 143.5],
  [142.5, 143.5],
  [150.5, 154.5],
  [152.5, 153.5],
  [152.5, 'infty'],
  [156.5, 'infty'],
  [157.5, 'infty'],
  [172.5, 'infty']],
 'largest_bars': [(114.5, 170.5),
  (120.5, 149.5),
  (103.5, 129.5),
  (107.5, 127.5),
  (126.5, 144.5)],
 'components': [0, 2440, 2949, 10053, 11727, 12178, 17131]}

In [6]:
for direction in directions:
    prof = cProfile.Profile()
    prof.enable()
    # code to profile
    for _ in range(10):
        process_direction((direction, vertices, edges))
    prof.disable()
    ps = pstats.Stats(prof).sort_stats("cumtime")
    ps.print_stats(10) 

         99615192 function calls in 22.373 seconds

   Ordered by: cumulative time
   List reduced from 56 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000   22.373    2.237 /var/folders/qd/fg0y6gy96rz748mcq02s5pcc0000gn/T/ipykernel_26339/2254060269.py:33(process_direction)
       10    0.009    0.001   14.868    1.487 /Users/kiyoshitakeuchi/Documents/GitHub/anu_rsop/jupyter_notebooks/3D-persistence/optimizations/persistent_homology.py:71(compute_persistence)
     3090    7.175    0.002   13.929    0.005 /Users/kiyoshitakeuchi/Documents/GitHub/anu_rsop/jupyter_notebooks/3D-persistence/optimizations/persistent_homology.py:119(compute_components)
       10    0.081    0.008    7.504    0.750 /Users/kiyoshitakeuchi/Documents/GitHub/anu_rsop/jupyter_notebooks/3D-persistence/optimizations/persistent_homology.py:59(__init__)
       10    0.053    0.005    7.419    0.742 /Users/kiyoshitakeuchi/Documents/GitHub/a